In [1]:
# Used for selecting the 6 best features per cluster. 
# Assumes data is vectorized + clustered.
# We're using simple f_regression score of each variable vs. the ALSFRS_score, and take the best 6. 

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression


In [3]:
proact_data = pd.read_csv('../train_data_vectorized.csv', sep = '|', index_col='SubjectID')
slope = pd.read_csv('../train_slope.csv', sep = '|', index_col='SubjectID')
clusters = pd.read_csv('../train_kmeans_clusters.csv', sep = '|', index_col='SubjectID')
X = pd.merge(proact_data, clusters, left_index = True, right_index = True)
Y = pd.merge(X, slope, left_index = True, right_index = True)

Y.groupby('cluster').mean()

,bp_diastolic,bp_systolic,height,pulse,respiratory_rate,temperature,weight,F,Asian,Black,Hispanic,Other,White,ALSFRS_slope
cluster,,,,,,,,,,,,,,
0,0.074945,0.144886,0.000275,0.026090,0.003552,-0.000335,0.005274,0.362795,0.009259,0.017677,0.005892,0.002525,0.950337,-0.729489
1,-0.251046,-0.579919,0.000184,-0.001990,0.008271,-0.000307,0.001740,0.355856,0.002252,0.011261,0.002252,0.000000,0.957207,-0.720036
2,-0.344796,-6.433254,0.000125,-2.271834,0.275022,-0.017081,0.059300,0.333333,0.000000,0.166667,0.000000,0.000000,0.833333,-0.787179


In [4]:
selector_per_cluster = {}
for c in clusters['cluster'].unique():
    seg_X, seg_Y = X[X['cluster'] == c], Y[Y['cluster'] == c]
    print c, seg_X.shape, seg_Y.shape
    selector = SelectKBest(f_regression, k=6)
    X_new = selector.fit_transform(seg_X, seg_Y['ALSFRS_slope'])
    print seg_X.columns[selector.get_support()]
    selector_per_cluster[c] = selector

1 (444, 14) (444, 15)
Index([u'bp_systolic', u'height', u'respiratory_rate', u'F', u'Asian',
       u'Black'],
      dtype='object')
0 (1188, 14) (1188, 15)
Index([u'bp_diastolic', u'bp_systolic', u'pulse', u'F', u'Asian', u'White'], dtype='object')
2 (6, 14) (6, 15)
Index([u'height', u'temperature', u'weight', u'F', u'Black', u'White'], dtype='object')


In [5]:
def calc(x):
    selector = selector_per_cluster[x['cluster']]
    d = {"feature_ " + str(i): v for i, v in enumerate(selector.transform(x)[0])}
    d['features_list'] = ';'.join(cur_X.columns[selector.get_support()])
    d['cluster'] = int(x['cluster'])
    return pd.Series(d)

for t in ['train', 'test']:
    cur_data = pd.read_csv('../' + t + '_data_vectorized.csv', sep = '|', index_col='SubjectID')
    cur_clusters = pd.read_csv('../' + t + '_kmeans_clusters.csv', sep = '|', index_col='SubjectID')
    cur_X = pd.merge(cur_data, cur_clusters, left_index = True, right_index = True)
    res = cur_X.apply(calc, axis = 1)
    res.to_csv('../' + t + '_selected_features.csv',sep='|')
    